In [3]:
import openpyxl
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver.support.ui import Select
from openpyxl import Workbook
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from colorama import Fore
import xlutils
import csv
import os
import pandas

In [9]:
Property_address_path="E:\Web Scrapping\Right Search\Data Capture Project - Responses\LA List with Addresses.xlsx"
wb=openpyxl.load_workbook(Property_address_path)
propertysheet=wb.active
rows_count=propertysheet.max_row
print(Fore.BLUE+"Total Addresses:"+Fore.LIGHTWHITE_EX+str(rows_count))
for count in range(14, rows_count + 1):
    Address = propertysheet.cell(row=count, column=1).value
    postalcode=findpostal1(Address)
    if postalcode == 0:
        print(Fore.LIGHTWHITE_EX+str(count)+Address+Fore.RED+" -Postal Code not found! Please Check the address...")
    elif(str(postalcode).isalnum()==False):
        print(Fore.LIGHTWHITE_EX+str(count)+Address+Fore.RED+" -"+str(postalcode))
    else:
        website=findweblink(postalcode)
        if(website==0):
            print(Fore.LIGHTWHITE_EX+Address+Fore.RED+" -Postal code of this address not found in the list")

Total Addresses:16
55 Clos Yr Eryr, Bridgend, CF35 6HE -Postal code of this address not found in the list
Cartref, Pentre Llyn, Llanilar, Aberystwyth, SY23 4NS -Postal code of this address not found in the list
79 Clayton Drive, Pontarddulais, Swansea, SA4 8AG -Postal code of this address not found in the list


In [6]:
def findpostal1(Address):#optional code if postalfinder website server is down we can use this but this is not an efficient one
    if ', ' in Address:
        x = Address.rsplit(", ")
        post=x[-1]
        y = post.rsplit(" ")
        if len(y) > 1:
            post=y[-2]
    else:
        if ',' in Address:
            x = Address.rsplit(",")
            post = x[-1]
            y = post.rsplit(" ")
            if len(y) > 1:
                post = y[-2]
        else:
            y =Address.rsplit(" ")
            post = y[-1]
            if len(y) > 1:
                post = y[-2]
    if post.isalnum():
        return post
    else:
        return 0

In [12]:
def findpostal(Address):
    try:
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get("https://postcodefinder.net/")
        time.sleep(5)
        wait = WebDriverWait(driver, 200)
        textinput= wait.until(EC.element_to_be_clickable((By.XPATH, "// *[ @ id = 'wrapper'] / div[1] / div[2] / form / input")))
        textinput.send_keys(Address)
        textsubmit= wait.until(EC.element_to_be_clickable((By.XPATH,"// *[ @ id = 'wrapper'] / div[1] / div[2] / form / button")))
        textsubmit.click()
        time.sleep(3)
        code= wait.until(EC.element_to_be_clickable((By.XPATH,"//*[@id='map']/div[1]/div[6]/div/div[1]/div/div[2]")))
        post=code.text.strip("Postcode: ")
        driver.close()
        x = post.rsplit(" ")
        if len(x)>1:
            post=x[-2]
        if(post=="unknown"):
            post=post+" "
        if(post.isalnum()==True):
            return post
        else:
            return 0
    except Exception as e:
        return e

In [20]:
def findweblink(postalcode):
    weblink=""
    Mapsheet_path = "E:\Web Scrapping\Right Search\LA List with Addresses_0825.xlsx"
    map_wb = openpyxl.load_workbook(Mapsheet_path)
    map_propertysheet = map_wb.active
    postalcode_maxrow = map_propertysheet.max_row
    for i in range(1,postalcode_maxrow+1):
        post_code = map_propertysheet.cell(row=i, column=1).value
        post_code_list=post_code.rsplit(",")
        post_code_list_len=len(post_code_list)
        for b in range(0,post_code_list_len):
            if(postalcode == post_code_list[b]):
                weblink = weblink +" " +map_propertysheet.cell(row=i, column=2).value
                return weblink
    if(weblink==""):
        return 0

In [32]:
def swansea(Address, website):
    try:
        driver = webdriver.Chrome(ChromeDriverManager().install())
        print(Fore.LIGHTYELLOW_EX+"Loading...")
        driver.get(website)
        driver.maximize_window()
        driver.implicitly_wait(10)
#         time.sleep(5)
        wait = WebDriverWait(driver, 150)
        textbox = wait.until(EC.element_to_be_clickable((By.ID, "simpleSearchString")))
#         time.sleep(8)
        textbox.click()
#         time.sleep(4)
        textbox.send_keys(Address)  # //*[@id="simpleSearchForm"]/div[3]/input[3]
        find = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@id='simpleSearchForm']/div[3]/input[3]")))
        find.click()  # find
#         time.sleep(5)
        results_container = driver.find_element_by_id("searchResultsContainer")
        ul = results_container.find_elements_by_tag_name("li")
        workbook = Workbook()
        sheet1 = workbook.active
        sheet1.title = "sheet 1"
        row_count = 1
        sheet1.cell(row=1, column=1).value = "Property Address:" + Address
        for row in range(0, len(ul)):
            row_count = row_count + 1
            sheet1.cell(row=row_count, column=1).value ="__________Result"+ str(row)+"__________"
            linklist = ul[row].find_element_by_tag_name("a")
            linklist.click()
            time.sleep(3)
            row_count=row_count+1
            tab_relatedCases = wait.until(EC.element_to_be_clickable((By.ID, "tab_relatedCases")))
            tab_relatedCases.click()
            Property = driver.find_element_by_xpath("//*[@id='Property']/ul/li/a")
            Property.click()
            tab_relatedCasesproparty = wait.until(EC.element_to_be_clickable((By.ID, "tab_relatedCases")))
            tab_relatedCasesproparty.click()
            Property = driver.find_element_by_id("Application")
            ul = Property.find_elements_by_tag_name("li") # //*[@id='Application']/ul/li[1]
#             breakpoint()
            for li in range(0, len(ul)):
                row_count = row_count + 1
                sheet1.cell(row=row_count, column=1).value ="__________RelatedCases"+ str(li)+"__________"
                unordered = ul[li].find_element_by_tag_name("a")
                unordered.click()
                time.sleep(3)
                sheet1.cell(row=row_count, column=1).value = "_____Summary_____"
                element4 = wait.until(EC.element_to_be_clickable((By.ID, "simpleDetailsTable")))
                table_rows = element4.find_elements_by_tag_name("tr")
                Listy = ["Reference", "Address", "Proposal", "Status", "Decision", "Decision Issued Date", "Appeal Status","Appeal Decision"]
                for t in table_rows:
                    thead = t.find_element_by_tag_name("th").text
                    if thead in Listy:
                        tbody = t.find_element_by_tag_name("td").text
                        row_count = row_count + 1
                        sheet1.cell(row=row_count, column=1).value = thead + ":" + tbody
                row_count = row_count + 1
                sheet1.cell(row=row_count, column=1).value = "_____Important Dates_____"
                element3 = wait.until(EC.element_to_be_clickable((By.ID, "subtab_dates")))
                element3.click()  # find
                element4 = wait.until(EC.element_to_be_clickable((By.ID, "simpleDetailsTable")))
                table_rows = element4.find_elements_by_tag_name("tr")
                Listy = ["Application Received Date", "Application Validated Date", "Decision Made Date","Decision Issued Date", "Decision Printed Date","Temporary Permission Expiry Date"]
                for t in table_rows:
                    thead = t.find_element_by_tag_name("th").text
                    if thead in Listy:
                        tbody = t.find_element_by_tag_name("td").text
                        row_count = row_count + 1
                        sheet1.cell(row=row_count, column=1).value = thead+":"+tbody
                workbook.save("E:/Web Scrapping/Right Search/Caerphilly/" + Address + ".xlsx")
                driver.back()
                driver.back()
                Property = driver.find_element_by_id("Application")
                ul = Property.find_elements_by_tag_name("li")
            driver.find_element_by_id("searchresultsback").click()
            results_container = driver.find_element_by_id("searchResultsContainer")
            ul = results_container.find_elements_by_tag_name("li")
        driver.close()
        #csv conversion
        data = pandas.read_excel("E:/Web Scrapping/Right Search/Caerphilly/" + Address + ".xlsx",engine='openpyxl')
        data.to_csv("E:/Web Scrapping/Right Search/Caerphilly/" + Address + ".csv", index=None, header=True)
        os.remove("E:/Web Scrapping/Right Search/Caerphilly/" + Address + ".xlsx")
        return 0
    except NoSuchElementException as e:
        return e

In [33]:
swansea('Ffynone Swansea', 'https://property.swansea.gov.uk/online-applications/')



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\yaraz\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


Loading...


WebDriverException: Message: chrome not reachable
  (Session info: chrome=99.0.4844.51)


In [ ]:
def merthyrTydfill(Address, website):
    try:
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(website)
        driver.maximize_window()
        print(Fore.LIGHTYELLOW_EX+"Loading...")
        driver.implicitly_wait(10)
        time.sleep(5)
        # select All
        wait = WebDriverWait(driver, 150)
        element = wait.until(EC.element_to_be_clickable((By.ID, "searchTypeStatus")))
        drp = Select(element)
        drp.select_by_visible_text("All")
#         time.sleep(8)
        # browser
        wait = WebDriverWait(driver, 150)
        element = wait.until(EC.element_to_be_clickable((By.ID, "simpleSearchString")))
#         time.sleep(8)
        element.click()
#         time.sleep(4)
        element.send_keys(Address)
        element2 = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@id='simpleSearchForm']/div[3]/input[3]")))
        element2.click()  # find
        tab_relatedCases = wait.until(EC.element_to_be_clickable((By.ID, "tab_relatedCases")))
        tab_relatedCases.click()
        Property = driver.find_element_by_xpath("//*[@id='Property']/ul/li/a")
        Property.click()
        tab_relatedCasesproperty = wait.until(EC.element_to_be_clickable((By.ID, "tab_relatedCases")))
        tab_relatedCasesproperty.click()
        Property = driver.find_element_by_id("Application")
        ul = Property.find_elements_by_tag_name("li")
        # for excel output
        workbook = Workbook()
        sheet1 = workbook.active
        sheet1.title = "sheet 1"
        row_count = 1
        sheet1.cell(row=row_count, column=1).value ="Property Address:"+Address
        for li in range(0, len(ul)):
            row_count = row_count + 1
            sheet1.cell(row=row_count, column=1).value ="__________RelatedCases"+ str(li)+"__________"
            unordered = ul[li].find_element_by_tag_name("a")
            unordered.click()
            time.sleep(3)
            sheet1.cell(row=row_count, column=1).value = "_____Summary_____"
            element4 = wait.until(EC.element_to_be_clickable((By.ID, "simpleDetailsTable")))
            table_rows = element4.find_elements_by_tag_name("tr")
            Listy = ["Reference", "Address", "Proposal", "Status", "Decision", "Decision Issued Date", "Appeal Status","Appeal Decision"]
            for tr in table_rows:
                thead = tr.find_element_by_tag_name("th").text
                if thead in Listy:
                    tbody = tr.find_element_by_tag_name("td").text
                    # pasting the output in excel
                    row_count = row_count + 1
                    sheet1.cell(row=row_count, column=1).value = thead + ":" + tbody
            row_count = row_count + 1
            sheet1.cell(row=row_count, column=1).value = "_____Important Dates_____"
            # extracting details
            element3 = wait.until(EC.element_to_be_clickable((By.ID, "subtab_dates")))
            element3.click()
            element4 = wait.until(EC.element_to_be_clickable((By.ID, "simpleDetailsTable")))
            table_rows = element4.find_elements_by_tag_name("tr")
            Listy=["Application Received Date","Application Validated Date","Decision Made Date","Decision Issued Date","Decision Printed Date","Temporary Permission Expiry Date"]
            breakpoint()
            for tr in table_rows:
                thead = tr.find_element_by_tag_name("th").text
                if thead in Listy:
                    tbody = tr.find_element_by_tag_name("td").text
                        # pasting the output in excel
                    row_count = row_count + 1
                    sheet1.cell(row=row_count, column=1).value =thead+":"+tbody
            address = Address.strip()
            workbook.save("E:/Web Scrapping/Right Search/Caerphilly/" + address + ".xlsx")
#         driver.find_element_by_id("searchresultsback").click()
#         results_container = driver.find_element_by_id("searchResultsContainer")
#         ul = results_container.find_elements_by_tag_name("li")
        driver.close()
        #csv conversion
        data = pandas.read_excel("E:/Web Scrapping/Right Search/Caerphilly/" + address + ".xlsx",engine='openpyxl')
        data.to_csv("E:/Web Scrapping/Right Search/Caerphilly/" + address + ".csv", index=None,header=True)
        os.remove("E:/Web Scrapping/Right Search/Caerphilly/" + address + ".xlsx")
        return 0
    except Exception as e:
        return e

In [ ]:
merthyrTydfill('mid glamorgan', 'https://publicaccess.merthyr.gov.uk/online-applications/')